# Tutorial for the Richardson-Gaudin model

In [73]:
import numpy as np
from moha.hamiltonians import HamRG, HamHeisenberg
from pyscf import fci

# Define the connectivity matrix
connectivity = np.array([[1, 1, 1, 1],
                         [1, 1, 1, 1],
                         [1, 1, 1, 1],
                         [1, 1, 1, 1]])

# connectivity = np.array([[1, 1],
#                          [1, 1]])


# Define the Hamiltonian parameters
J = -3
J_eq = J * connectivity
mu = J * np.ones(4)

# Define the Hamiltonian
ham = HamRG(J_eq=J_eq, mu=mu)
e0 = ham.generate_zero_body_integral()

h1 = ham.generate_one_body_integral(dense=True, basis='spatial basis')
h2 = ham.generate_two_body_integral(dense=True, basis='spatial basis', sym=4)

h2_ch = np.einsum('ijkl->ikjl', h2)


n_sites = 4
occ = (n_sites//2, n_sites//2)
# e0 += -3 * 2/2

e, ci = fci.direct_spin0.kernel(h1, 2*h2_ch, nelec=occ, norb=n_sites, nroots=6, ecore=e0)


print(e, e0)

# compute rdms
rdm1, rdm2 = fci.direct_spin0.make_rdm12(ci[0], norb=n_sites, nelec=occ)

# compute the repulsion energy
E_rep = e[0] - np.sum(h1*rdm1)
print("total energy: ", e)
print("repulsion energy: ", E_rep)

[-1.80000000e+001 -6.00000000e+000 -2.75592582e-307] 0.0
total energy:  [-1.80000000e+001 -6.00000000e+000 -2.75592582e-307]
repulsion energy:  -17.999999999999986


In [74]:
h1

matrix([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [75]:
for i, j, k, l in zip(*np.where(h2 != 0)):
    elem = h2[i, j, k, l]
    print(i, j, k, l,": ", elem)

print("one-body integral: ", h1)


0 0 0 0 :  -1.5
0 0 1 1 :  -1.5
0 0 2 2 :  -1.5
0 0 3 3 :  -1.5
1 1 0 0 :  -1.5
1 1 1 1 :  -1.5
1 1 2 2 :  -1.5
1 1 3 3 :  -1.5
2 2 0 0 :  -1.5
2 2 1 1 :  -1.5
2 2 2 2 :  -1.5
2 2 3 3 :  -1.5
3 3 0 0 :  -1.5
3 3 1 1 :  -1.5
3 3 2 2 :  -1.5
3 3 3 3 :  -1.5
one-body integral:  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [76]:
e0

0.0